In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
# Load image and convert to base64
import base64
from pathlib import Path

# Load the test image
image_path = Path("test_adnane.jpg")
with open(image_path, "rb") as image_file:
    image_data = image_file.read()

# Convert to base64
image_base64 = base64.b64encode(image_data).decode("utf-8")
print(f"Image loaded successfully! Size: {len(image_data)} bytes")
print(f"Base64 length: {len(image_base64)} characters")



Image loaded successfully! Size: 1288035 bytes
Base64 length: 1717380 characters


In [3]:
type(image_base64)

str

In [14]:
# Test the API endpoint
import requests
import json

# API endpoint URL (adjust if running on different host/port)
api_url = "http://localhost:8000/fashion-workflow"

# Test data
test_data = {
    "base64_image": image_base64,  # Using the base64 from previous cell
    "user_input": "What should I wear to a job interview?"
}

# Make the API request
try:
    print("Sending request to API...")
    response = requests.post(api_url, json=test_data)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    
    if response.status_code == 200:
        result = response.json()
        print("✅ API call successful!")
        print(f"Success: {result.get('success')}")
        print(f"Text Response: {result.get('text', 'No text')[:200]}...")
        print(f"Number of images: {len(result.get('images', []))}")
        
        if result.get('error_message'):
            print(f"Error message: {result.get('error_message')}")
    else:
        print(f"❌ API call failed with status {response.status_code}")
        print(f"Response: {response.text}")
        
except requests.exceptions.ConnectionError:
    print("❌ Connection failed. Make sure the API server is running on localhost:8000")
    print("Start the server with: python api.py")
except Exception as e:
    print(f"❌ Error: {str(e)}")


Sending request to API...
Status Code: 200
Response Headers: {'date': 'Sat, 11 Oct 2025 14:50:11 GMT', 'server': 'uvicorn', 'content-length': '3653469', 'content-type': 'application/json'}
✅ API call successful!
Success: True
Text Response: For the modern woman who commands attention without sacrificing grace, we’ve curated two looks embodying understated power. The first delivers polished professionalism with a navy blazer and wide-leg ...
Number of images: 2


In [17]:
# Save images to files instead of displaying them
from PIL import Image
import io
import base64
import os

images = result.get("images", [])
print(f"Number of images to save: {len(images)}")

if images:
    # Create a directory for saved images if it doesn't exist
    output_dir = "saved_images"
    os.makedirs(output_dir, exist_ok=True)
    
    for i, image_data in enumerate(images, 1):
        print(f"\nProcessing image {i}...")
        print(f"Image structure: {image_data}")
        
        # Try different possible keys for the base64 data
        img_base64 = None
        possible_keys = ["base64", "image", "data", "content"]
        
        for key in possible_keys:
            if key in image_data:
                img_base64 = image_data[key]
                print(f"Found image data under key: {key}")
                break
        
        if img_base64:
            try:
                # Decode base64 and save as image
                img_bytes = base64.b64decode(img_base64)
                img = Image.open(io.BytesIO(img_bytes))
                
                # Save the image
                filename = f"image_{i}.jpg"
                filepath = os.path.join(output_dir, filename)
                img.save(filepath, "JPEG", quality=95)
                
                print(f"✅ Saved image {i} as: {filepath}")
                print(f"   Description: {image_data.get('description', 'No description')}")
                print(f"   Image size: {img.size}")
                
            except Exception as e:
                print(f"❌ Error saving image {i}: {e}")
        else:
            print(f"❌ No base64 image data found in image {i}")
            print(f"   Available keys: {list(image_data.keys())}")
    
    print(f"\n🎉 All images saved to '{output_dir}' directory!")
    print(f"Files created: {[f'image_{i}.jpg' for i in range(1, len(images) + 1)]}")
    
else:
    print("No images found in result.")


Number of images to save: 2

Processing image 1...
Image structure: {'base64': '...', 'description': 'Generated outfit image'}
Found image data under key: base64
✅ Saved image 1 as: saved_images\image_1.jpg
   Description: Generated outfit image
   Image size: (1248, 832)

Processing image 2...
Image structure: {'base64': '...', 'description': 'Generated outfit image'}
Found image data under key: base64
✅ Saved image 2 as: saved_images\image_2.jpg
   Description: Generated outfit image
   Image size: (1248, 832)

🎉 All images saved to 'saved_images' directory!
Files created: ['image_1.jpg', 'image_2.jpg']
